## Data Wrangling Challenge

In [7]:
import pandas as pd
mypath = "/Users/jurajkapasny/Drive/Data/Car_crashes/"

In [46]:
data = pd.read_csv("~/Downloads/VT_VehicleAnimal_Collisions__2006.csv", 
                   na_values=['', ' '],
                   parse_dates=["DATE_"]
                  )
data["MONTH_"] = data.DATE_.dt.month
data.dropna(subset=['X', 'Y'], inplace=True)
# creation of variable with lon and lat together
data['ll'] = data['Y'].astype(str) + ',' + data['X'].astype(str)
print(data.shape)
data.head()

(2244, 23)


,X,Y,OBJECTID,MSRI_CODE,MSRI_DESCR,DATE_,TOWN_ID,ROUTE_DES,ROUTE,BEGIN_MM,...,TOWN,RT_TOWN,RT_NUM,DAY_,MONTH_,YEAR_,REP_AGEN,LOCATION,YEAR_INT,ll
4,-72.989424,42.833309,5,Deer,Deer,2004-04-16 00:00:00+00:00,209.0,VT 8,008,0.35,...,READSBORO,NaN,NaN,NaN,4.0,2004,AOT,NaN,2004,"42.83330874047745,-72.98942388725152"
5,-72.375495,43.581005,6,Deer RK AB,"Deer, RoadKill, Adult Buck",2005-03-22 00:00:00+00:00,1409.0,US 5,005,5.00,...,HARTLAND,NaN,NaN,NaN,3.0,2005,AOT,NaN,2005,"43.581005107295645,-72.37549549789222"
6,-72.840257,43.235023,7,Lg Bird,"Large Bird (hawk, owl, turkey, waterfowl)",2004-11-15 00:00:00+00:00,1310.0,VT 11,011,0.50,...,LONDONDERRY,NaN,NaN,NaN,11.0,2004,AOT,NaN,2004,"43.23502288333559,-72.8402568531839"
7,-72.830991,44.783002,8,Otter,Otter,2004-10-18 00:00:00+00:00,601.0,VT 36,036,1.00,...,BAKERSFIELD,NaN,NaN,NaN,10.0,2004,AOT,NaN,2004,"44.78300220380285,-72.83099079061695"
9,-72.514595,44.242323,10,Beaver,Beaver,2005-04-22 00:00:00+00:00,1207.0,US 2,002,0.19,...,EAST MONTPEL,NaN,NaN,NaN,4.0,2005,AOT,NaN,2005,"44.24232301744294,-72.51459452575021"


In [9]:
#set the keys
foursquare_id = os.environ['']
foursquare_secret = os.environ['']

In [10]:
from foursquare import Foursquare

In [11]:
client = Foursquare(client_id = foursquare_id,
                    client_secret = foursquare_secret)

In [14]:
this_crash = data.iloc[0,:]

In [21]:
town = this_crash["TOWN"] + ",VT"

In [26]:
response = client.venues.search({"near" : town,
                                         "limit" : 50,
                                         "intent" : "browse",
                                         "radius" : 5000,
                                         "categoryId" : "4d4b7105d754a06376d81259" })

In [36]:
# Because of limitations in requests, we are going to find number of bars and venues for each town only
towns = list(data["TOWN"].value_counts().index)
print(len(towns))

326


In [37]:
number_of_bars = {}
ind = 0
for town in towns:
    if ind % 50 == 0:
        print(ind)
    try:
        # we have collission in Vermont (VT) so we add VT so API is sure which town we mean.
        town = town + ",VT"
        response = client.venues.search({"near" : town,
                                         "limit" : 50,
                                         "intent" : "browse",
                                         "radius" : 5000,
                                         # this category is for nightlife: https://developer.foursquare.com/docs/build-with-foursquare/categories/
                                         "categoryId" : "4d4b7105d754a06376d81259"})
        number_of_bars[town] = len(response["venues"])
    except Exception as e:
        print(e)
        if e == "Quota exceeded":
            print("exceeded quota: waiting for an hour")
            time.sleep(3600)
        number_of_bars[town] = -1
    ind += 1

0
Couldn't geocode param near: WARREN'S GOR,VT
Couldn't geocode param near: EAST MONTPEL,VT
Couldn't geocode param near: ST. JOHNSBUR,VT
50
Couldn't geocode param near: Sherburne,VT
Couldn't geocode param near: ST. ALBANS T,VT
100
Couldn't geocode param near: WEATHERSFIEL,VT
Couldn't geocode param near: BARRE TOWN,VT
Couldn't geocode param near: RUTLAND TOWN,VT
Couldn't geocode param near: St. Johnsbur,VT
150
Couldn't geocode param near: WARREN GORE,VT
Couldn't geocode param near: NEWPORT TOWN,VT
200
250
Couldn't geocode param near: Weathersfiel,VT
Couldn't geocode param near: Ornage,VT
Couldn't geocode param near: East Montpel,VT
Couldn't geocode param near: BANTON,VT
300
Couldn't geocode param near: BUELL'S GORE,VT
Couldn't geocode param near: ENOSBURG,VT
Couldn't geocode param near: SHERBURNE,VT
Couldn't geocode param near: Warren Gore,VT


In [54]:
df_number_of_bars = pd.DataFrame(number_of_bars, index=["number_of_bars"]).transpose()

In [55]:
df_number_of_bars.index = [town[:-3] for town in df_number_of_bars.index]

In [56]:
df_number_of_bars.head()

,number_of_bars
LUNENBURG,0
SHEFFIELD,0
CANAAN,2
CONCORD,0
NORTON,0


In [57]:
data = data.merge(df_number_of_bars, how="inner", left_on = ["TOWN"], right_index=True)

In [76]:
data.head()

,X,Y,OBJECTID,MSRI_CODE,MSRI_DESCR,DATE_,TOWN_ID,ROUTE_DES,ROUTE,BEGIN_MM,...,RT_TOWN,RT_NUM,DAY_,MONTH_,YEAR_,REP_AGEN,LOCATION,YEAR_INT,ll,number_of_bars
4,-72.989424,42.833309,5,Deer,Deer,2004-04-16 00:00:00+00:00,209.0,VT 8,008,0.35,...,NaN,NaN,NaN,4.0,2004,AOT,NaN,2004,"42.83330874047745,-72.98942388725152",0
47,-72.985410,42.826518,48,Deer,Deer,2004-03-14 00:00:00+00:00,209.0,VT 100,100,2.40,...,NaN,NaN,NaN,3.0,2004,AOT,NaN,2004,"42.826518294188965,-72.98540968358323",0
69,-72.987662,42.828544,70,Deer RK AB,"Deer, RoadKill, Adult Buck",2005-11-22 00:00:00+00:00,209.0,VT 100,100,2.22,...,NaN,NaN,NaN,11.0,2005,AOT,NaN,2005,"42.82854445067909,-72.98766171275436",0
106,-72.987662,42.828544,107,Deer RK AB,"Deer, RoadKill, Adult Buck",2005-11-04 00:00:00+00:00,209.0,VT 100,100,2.22,...,NaN,NaN,NaN,11.0,2005,AOT,NaN,2005,"42.82854445067909,-72.98766171275436",0
173,-73.000182,42.826541,174,Moose,Moose,2005-11-07 00:00:00+00:00,209.0,VT 100,100,1.50,...,NaN,NaN,NaN,11.0,2005,AOT,NaN,2005,"42.82654059828223,-73.00018178424375",0


## Virtual Crossing

In [97]:
import requests

In [79]:
data["DATE_"] = data["DATE_"].dt.date

In [85]:
# we group by day and town to limit number of queries for API
info_for_weather_api = data.groupby(["DATE_","TOWN"]).size().reset_index()

In [84]:
# this sample URL is from the documentation here: https://www.visualcrossing.com/resources/documentation/weather-api/weather-api-documentation/#history
api_key = os.environ['']
start_date = "1971-06-18"
end_date = "1971-06-18"
location = ""
url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=24&startDateTime={start_date}T00:00:00&endDateTime={end_date}T00:00:00&day&location={location}&key={api_key}"

In [104]:
weather_results = []
for row in info_for_weather_api.iterrows():
    start_date = list(row[1])[0]
    start_date = list(row[1])[0]
    location = list(row[1])[1] + ",VT,US"
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=24&startDateTime={start_date}T00:00:00&endDateTime={end_date}T00:00:00&day&location={location}&key={api_key}"
    res = requests.get(url)
    weather_results.append(res.text)

In [125]:
# parsing the text returned by the API
weather = []
for row in weather_results:
    try:
        weather.append(row.split('"')[-2])
    except IndexError:
        weather.append(None)

In [134]:
info_for_weather_api["weather"] = weather

In [137]:
# we weren't able to get all the information about the weather because of the requests
info_for_weather_api.head()

,DATE_,TOWN,0,weather
0,1971-06-18,Glover,1,
1,1971-07-07,Mt Holly,1,
2,1973-07-01,Readsboro,1,"Rain, Partially cloudy"
3,1973-08-25,Eden,1,"Rain, Clear"
4,1973-09-25,Readsboro,1,"Rain, Partially cloudy"


In [139]:
data = data.merge(info_for_weather_api, how = "inner", left_on = ["DATE_","TOWN"], right_on=["DATE_","TOWN"])

In [141]:
len(data)

2205

In [143]:
data.tail()

,X,Y,OBJECTID,MSRI_CODE,MSRI_DESCR,DATE_,TOWN_ID,ROUTE_DES,ROUTE,BEGIN_MM,...,DAY_,MONTH_,YEAR_,REP_AGEN,LOCATION,YEAR_INT,ll,number_of_bars,0,weather
2200,-72.749425,44.921173,1847,Moose,NaN,1985-05-11,NaN,NaN,NaN,0.0,...,11.0,5.0,1985,F&W,DAIRY CENTER,1985,"44.92117296101513,-72.74942506180635",-1,1,"Rain, Partially cloudy"
2201,-73.082234,44.725994,1848,Moose,NaN,1997-09-12,NaN,NaN,NaN,0.0,...,12.0,9.0,1997,F&W,I-89 SBL MM 109,1997,"44.72599408182294,-73.0822336071397",2,1,None
2202,-72.709793,44.643004,1902,Moose,NaN,1991-09-13,NaN,NaN,NaN,0.0,...,13.0,9.0,1991,F&W,RT 15 AND HOGBACK RD,1991,"44.64300421293702,-72.70979266020093",2,1,Partially cloudy
2203,-72.665773,44.632526,1903,Moose,NaN,1997-04-19,NaN,NaN,NaN,0.0,...,19.0,4.0,1997,F&W,Rte. 15 Lehouiller Farm,1997,"44.63252635217222,-72.66577330285902",2,1,None
2204,-72.742990,43.965952,1915,Moose,NaN,1998-11-27,NaN,NaN,NaN,0.0,...,27.0,11.0,1998,F&W,RT 12A 0.7 mi South of W. Braintree,1998,"43.96595164264534,-72.74298958433192",1,1,None
